# 実習①　データ前処理


## Rによる処理の基礎

### パイプ処理入門

パイプ処理：データを効率的に処理するための枠組み



+ 通常の処理：ひとつの処理が完全に終わったら次の処理へと逐次的に処理する


xというデータにf1, f2, ... fnの関数を順に適用したときのイメージ


```
fn(fn-1(...(f2(f1())
```

+ パイプ処理：終わった部分から徐々に次の処理に流していき、並列的に処理する

ここでパイプ演算子を|として、

```
f1(x) | f2 | ... | fn
```

linuxなどで処理するときに重宝する



Rでは"dplyr"パッケージを使う

dplyrの使い方についてはRStudioチームのチートシートが参考になる

https://www.rstudio.com/resources/cheatsheets/

In [1]:
library("dplyr")


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



組み込みのToothGrowthデータで実演

In [2]:
# data description
?ToothGrowth

ToothGrowth              package:datasets              R Documentation

_T_h_e _E_f_f_e_c_t _o_f _V_i_t_a_m_i_n _C _o_n _T_o_o_t_h _G_r_o_w_t_h _i_n _G_u_i_n_e_a _P_i_g_s

_D_e_s_c_r_i_p_t_i_o_n:

     The response is the length of odontoblasts (cells responsible for
     tooth growth) in 60 guinea pigs.  Each animal received one of
     three dose levels of vitamin C (0.5, 1, and 2 mg/day) by one of
     two delivery methods, orange juice or ascorbic acid (a form of
     vitamin C and coded as 'VC').

_U_s_a_g_e:

     ToothGrowth
     
_F_o_r_m_a_t:

     A data frame with 60 observations on 3 variables.

       [,1]  len   numeric  Tooth length                
       [,2]  supp  factor   Supplement type (VC or OJ). 
       [,3]  dose  numeric  Dose in milligrams/day      
      
_S_o_u_r_c_e:

     C. I. Bliss (1952) _The Statistics of Bioassay_.  Academic Press.

_R_e_f_e_r_e_n_c_e_s:

     McNeil, D. R. (1977) _Interactive Data Analysis_.  New York:
     Wiley.

     Crampton, E. W. (1947) The growth of the odontoblast of the
     incisor teeth as a criterion of vitamin C intake of the guinea
     pig. _The Journal of Nutrition_ *33(5)*: 491-504.  <URL:
     http://jn.nutrition.org/content/33/5/491.full.pdf>

_E_x_a_m_p_l_e_s:

     require(graphics)
     coplot(len ~ dose | supp, data = ToothGrowth, panel = panel.smooth,
            xlab = "ToothGrowth data: length vs dose, given type of supplement")

In [3]:
# ToothGrowth dataset

##[,1]	len 	numeric	Tooth length
##[,2]	supp	factor	Supplement type (VC or OJ).
##[,3]	dose	numeric	Dose in milligrams/day

# 普通にhead()
head(ToothGrowth)


,len,supp,dose
,<dbl>,<fct>,<dbl>
1,4.2,VC,0.5
2,11.5,VC,0.5
3,7.3,VC,0.5
4,5.8,VC,0.5
5,6.4,VC,0.5
6,10.0,VC,0.5


In [4]:
# これがこう書ける
ToothGrowth %>% head()


,len,supp,dose
,<dbl>,<fct>,<dbl>
1,4.2,VC,0.5
2,11.5,VC,0.5
3,7.3,VC,0.5
4,5.8,VC,0.5
5,6.4,VC,0.5
6,10.0,VC,0.5


In [5]:
# 行の指定
# subset / filter
# head(subset(ToothGrowth, supp == "OJ"))

ToothGrowth %>% filter(supp == "OJ") %>% head()

,len,supp,dose
,<dbl>,<fct>,<dbl>
1,15.2,OJ,0.5
2,21.5,OJ,0.5
3,17.6,OJ,0.5
4,9.7,OJ,0.5
5,14.5,OJ,0.5
6,10.0,OJ,0.5


In [6]:
# 列の指定
# select
# len列とsupp列を、len列を「length」、supp列を「supp_type」として出力

ToothGrowth %>% select(length=len, supp_type=supp) %>% head()

,length,supp_type
,<dbl>,<fct>
1,4.2,VC
2,11.5,VC
3,7.3,VC
4,5.8,VC
5,6.4,VC
6,10.0,VC


### テキスト処理の基礎
#### stringrパッケージ

In [7]:
# libraryの読み込み
library('stringr')


+ 検索
    + str_subset() : 指定されたキーワードが含まれている文字列を返す
    + str_detect() : 指定されたキーワードが含まれていればTRUE, そうでなければFALSEを返す
    + grep: Rの標準の関数。ヒットしたベクトルの位置を返す
+ 置換
    + str_replace() : 指定された検索語を置換語で置き換える
    + sub: Rの標準の関数。検索語を置換語で置き換える



In [8]:
# 月名のリスト
(months = month.name) 

[1] "January"   "February"  "March"     "April"     "May"       "June"     
 [7] "July"      "August"    "September" "October"   "November"  "December"

In [9]:
# "r"を含む月名を抜き出す
# str_subset
months %>% str_subset('r')

[1] "January"   "February"  "March"     "April"     "September" "October"  
[7] "November"  "December"

In [10]:
# str_detect では真偽値が返る
months %>% str_detect('r')

[1]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE

### 正規表現

**正規表現** *regular expression* :柔軟に文字列を処理するための記法

※言語ごとに記法に差異があるので注意する


#### 基本

|記号|説明|
|---|---|
|^|行頭|
|$|行頭|
|.|任意の1文字|
|\||または|

#### エスケープシーケンス

|記号|説明|
|---|---|
|\\\\d|数字(digits)|
|\\\\w|単語に使われる文字・数字・アンダーバー(words)|
|\\\\s|空白(spaces)|
|\\n|改行(newline)|
|\\t|タブ(tab)|



#### 量指定子

|記号|説明|使用例|結果|
|---|---|---|---|
|*|0回以上の繰り返し|||
|+|1回以上の繰り返し|\\\\d+|1回以上の数字の連続|
|?|0または1回|https?|httpまたはhttps|

#### エスケープ文字

正規表現など特殊な機能を持つ文字については\\\\(バックスラッシュまたは円記号２つ)を直前につける


#### 制御

|記号|説明|使用例|結果|
|---|---|---|---|
|[ ]|括弧内に含まれる文字にマッチ|[abcde]|abcdeのいずれか1文字|
|[^ ]|括弧内に含まれない文字にマッチ|[^abcde]|abcde以外の文字1文字|
|( )|グループ化|(abcde)|「abcde」という連続する文字のグループ|



    

#### その他の文字クラス

|記号|説明|
|---|---|
|[:alpha:]|アルファベット|
|[:lower:]|小文字アルファベット|
|[:upper:]|大文字アルファベット|
|[:digit:]|数字|
|[:alnum:]|アルファベット＋数字|
|[:punct:]|記号|
|[:graph:]|数字＋記号＋アルファベット|
|[:blank:]|空白、タブ|


### 正規表現の練習


In [11]:
# 冒頭三文字
'abcdefg' %>% str_extract('^...')

[1] "abc"

In [12]:
# 普通の文字列
'abcdefg' %>% str_extract('bcd')

[1] "bcd"

In [13]:
# 末尾三文字
'abcdefg' %>% str_extract('...$')

[1] "efg"

In [14]:
# abまたはefにマッチしたものをすべて
'abcdefg' %>% str_extract_all('ab|ef')

[[1]]
[1] "ab" "ef"

In [15]:
# 例文
sentence = 'googleのトップページはhttps://www.google.co.jp/です'

In [16]:
# 冒頭の英字のみを抜き出す
sentence %>% str_extract('^[A-z]+')

[1] "google"

In [17]:
# urlを抜き出してみる
sentence %>% str_extract('https?[A-z:/\\.]*')

[1] "https://www.google.co.jp/"

In [18]:
# 例文
sentence = "computational social sciences"

In [19]:
# 単語(アルファベットの連続)をすべて抜き出す
sentence %>% str_extract_all('[A-z]+') 

[[1]]
[1] "computational" "social"        "sciences"

In [20]:
# sで始まる単語をすべて抜き出す
sentence %>% str_extract_all('s[A-z]*')

[[1]]
[1] "social"   "sciences"

In [21]:
# -alで終わる単語をすべて抜き出す
sentence %>% str_extract_all('[A-z]*al')

[[1]]
[1] "computational" "social"

In [22]:
# 母音をすべて抜き出す
sentence %>% str_extract_all('[aiueo]')

[[1]]
 [1] "o" "u" "a" "i" "o" "a" "o" "i" "a" "i" "e" "e"

In [23]:
# 母音をすべて抜き出す
sentence %>% str_extract_all('[aiueo]')

[[1]]
 [1] "o" "u" "a" "i" "o" "a" "o" "i" "a" "i" "e" "e"

### その他処理

#### in 演算子

%in% を使うと要素がベクトルの中に含まれるかを判定できる



In [24]:
c(1,2,3,4,5) %in% c(1,5) # 要素が1または5であればTRUE

[1]  TRUE FALSE FALSE FALSE  TRUE

## RMecabの基礎

+ MeCab: 形態素解析エンジン
+ RMeCab: MeCabをR上で使うためのパッケージ


In [25]:
#install.packages('RMeCab', repos='http://rmecab.jp/R')

In [26]:
library('RMeCab')

In [27]:
RMeCabC('今日は本を読んだ') %>% unlist()

名詞   助詞   名詞   助詞   動詞 助動詞 
"今日"   "は"   "本"   "を" "読ん"   "だ"

In [28]:
test = RMeCabC('今日は本を読んだ') %>% unlist()

In [29]:
# 結果から名詞と動詞を抜き出す

test[names(test) %in% c('名詞', '動詞')]

名詞   名詞   動詞 
"今日"   "本" "読ん"

In [30]:
names(test) %in% c('名詞', '動詞')

[1]  TRUE FALSE  TRUE FALSE  TRUE FALSE

In [31]:
# 第二引数に1を入力すると、動詞の原形になる

RMeCabC('今日は本を読んだ', 1) %>% unlist()

名詞   助詞   名詞   助詞   動詞 助動詞 
"今日"   "は"   "本"   "を" "読む"   "だ"

## クリーニングの実際

例文：

「やまいり？やまはいって木を丸める？」

**「切ってきて、丸めて、家の屋根にしげておくん。それはてんかごめんけどどこのやまいってきってきてもよいということになっちょった。」**


In [32]:
text = '切ってきて、丸めて、家の屋根にしげておくん。それはてんかごめんけどどこのやまいってきってきてもよいということになっちょった。'

In [33]:
RMeCabC(text)

[[1]]
  動詞 
"切っ" 

[[2]]
助詞 
"て" 

[[3]]
動詞 
"き" 

[[4]]
助詞 
"て" 

[[5]]
記号 
"、" 

[[6]]
  動詞 
"丸め" 

[[7]]
助詞 
"て" 

[[8]]
記号 
"、" 

[[9]]
名詞 
"家" 

[[10]]
助詞 
"の" 

[[11]]
  名詞 
"屋根" 

[[12]]
助詞 
"に" 

[[13]]
形容詞 
"しげ" 

[[14]]
助詞 
"て" 

[[15]]
  動詞 
"おく" 

[[16]]
助動詞 
  "ん" 

[[17]]
記号 
"。" 

[[18]]
  名詞 
"それ" 

[[19]]
助詞 
"は" 

[[20]]
    名詞 
"てんか" 

[[21]]
  感動詞 
"ごめん" 

[[22]]
接続詞 
"けど" 

[[23]]
  名詞 
"どこ" 

[[24]]
助詞 
"の" 

[[25]]
助詞 
"や" 

[[26]]
    動詞 
"まいっ" 

[[27]]
助詞 
"て" 

[[28]]
  動詞 
"きっ" 

[[29]]
助詞 
"て" 

[[30]]
動詞 
"き" 

[[31]]
助詞 
"て" 

[[32]]
助詞 
"も" 

[[33]]
形容詞 
"よい" 

[[34]]
    助詞 
"という" 

[[35]]
  名詞 
"こと" 

[[36]]
助詞 
"に" 

[[37]]
  動詞 
"なっ" 

[[38]]
  名詞 
"ちょ" 

[[39]]
動詞 
"っ" 

[[40]]
助動詞 
  "た" 

[[41]]
記号 
"。"

表現を変える

'切ってきて、丸めて、家の屋根にしげておくん。それはてんかごめんけどどこのやまいってきってきてもよいということになっちょった。'

'切ってきて、丸めて、家の屋根に す げておくん。それは 天下御免 けどどこの山 に行って切ってきてもよいということになっていた。'

In [34]:
text = '切ってきて、丸めて、家の屋根にすげておくん。それは天下御免でどこの山に行って切ってきてもよいということになっていた。'

In [35]:
RMeCabC(text)

[[1]]
  動詞 
"切っ" 

[[2]]
助詞 
"て" 

[[3]]
動詞 
"き" 

[[4]]
助詞 
"て" 

[[5]]
記号 
"、" 

[[6]]
  動詞 
"丸め" 

[[7]]
助詞 
"て" 

[[8]]
記号 
"、" 

[[9]]
名詞 
"家" 

[[10]]
助詞 
"の" 

[[11]]
  名詞 
"屋根" 

[[12]]
助詞 
"に" 

[[13]]
  動詞 
"すげ" 

[[14]]
助詞 
"て" 

[[15]]
  動詞 
"おく" 

[[16]]
助動詞 
  "ん" 

[[17]]
記号 
"。" 

[[18]]
  名詞 
"それ" 

[[19]]
助詞 
"は" 

[[20]]
  名詞 
"天下" 

[[21]]
  名詞 
"御免" 

[[22]]
助詞 
"で" 

[[23]]
  名詞 
"どこ" 

[[24]]
助詞 
"の" 

[[25]]
名詞 
"山" 

[[26]]
助詞 
"に" 

[[27]]
  動詞 
"行っ" 

[[28]]
助詞 
"て" 

[[29]]
  動詞 
"切っ" 

[[30]]
助詞 
"て" 

[[31]]
動詞 
"き" 

[[32]]
助詞 
"て" 

[[33]]
助詞 
"も" 

[[34]]
形容詞 
"よい" 

[[35]]
    助詞 
"という" 

[[36]]
  名詞 
"こと" 

[[37]]
助詞 
"に" 

[[38]]
  動詞 
"なっ" 

[[39]]
助詞 
"て" 

[[40]]
動詞 
"い" 

[[41]]
助動詞 
  "た" 

[[42]]
記号 
"。"

In [36]:
# 辞書登録前
RMeCabC('まあそういうことで、余分な話はいいけん。')

[[1]]
  副詞 
"まあ" 

[[2]]
    連体詞 
"そういう" 

[[3]]
  名詞 
"こと" 

[[4]]
助動詞 
  "で" 

[[5]]
記号 
"、" 

[[6]]
  名詞 
"余分" 

[[7]]
助動詞 
  "な" 

[[8]]
名詞 
"話" 

[[9]]
助詞 
"は" 

[[10]]
形容詞 
"いい" 

[[11]]
助詞 
"け" 

[[12]]
助詞 
"ん" 

[[13]]
記号 
"。"

In [37]:
getwd()

[1] "C:/Users/satoc/projects/lecture2019"

In [38]:
# 辞書登録後
RMeCabC('まあそういうことで、余分な話はいいけん。', dic='dict/example.dic')

[[1]]
  副詞 
"まあ" 

[[2]]
    連体詞 
"そういう" 

[[3]]
  名詞 
"こと" 

[[4]]
助動詞 
  "で" 

[[5]]
記号 
"、" 

[[6]]
  名詞 
"余分" 

[[7]]
助動詞 
  "な" 

[[8]]
名詞 
"話" 

[[9]]
助詞 
"は" 

[[10]]
形容詞 
"いい" 

[[11]]
  助詞 
"けん" 

[[12]]
記号 
"。"

In [39]:
# 辞書登録前
RMeCabC('そこに来ちょる。')

[[1]]
  名詞 
"そこ" 

[[2]]
助詞 
"に" 

[[3]]
動詞 
"来" 

[[4]]
  名詞 
"ちょ" 

[[5]]
助動詞 
  "る" 

[[6]]
記号 
"。"

In [40]:
# 辞書登録後
RMeCabC('そこに来ちょる。', dic='dict/example.dic')


[[1]]
  名詞 
"そこ" 

[[2]]
助詞 
"に" 

[[3]]
動詞 
"来" 

[[4]]
    動詞 
"ちょる" 

[[5]]
記号 
"。"

In [41]:
# 活用語は活用ごとに登録しないといけない
RMeCabC('そこに来ちょった。', dic='dict/example.dic')

[[1]]
  名詞 
"そこ" 

[[2]]
助詞 
"に" 

[[3]]
動詞 
"来" 

[[4]]
  名詞 
"ちょ" 

[[5]]
動詞 
"っ" 

[[6]]
助動詞 
  "た" 

[[7]]
記号 
"。"

In [42]:
text = 'しまあたのぉ、ここへ今日イノシシの肉が家にあったけん持ってくーだった'

In [43]:
RMeCabC(text)

[[1]]
動詞 
"し" 

[[2]]
  副詞 
"まあ" 

[[3]]
助動詞 
  "た" 

[[4]]
助詞 
"の" 

[[5]]
名詞 
"ぉ" 

[[6]]
記号 
"、" 

[[7]]
  名詞 
"ここ" 

[[8]]
助詞 
"へ" 

[[9]]
  名詞 
"今日" 

[[10]]
      名詞 
"イノシシ" 

[[11]]
助詞 
"の" 

[[12]]
名詞 
"肉" 

[[13]]
助詞 
"が" 

[[14]]
名詞 
"家" 

[[15]]
助詞 
"に" 

[[16]]
  動詞 
"あっ" 

[[17]]
助動詞 
  "た" 

[[18]]
  動詞 
"けん" 

[[19]]
  動詞 
"持っ" 

[[20]]
助詞 
"て" 

[[21]]
動詞 
"く" 

[[22]]
名詞 
"ー" 

[[23]]
助動詞 
"だっ" 

[[24]]
助動詞 
  "た"

In [44]:
text = 'しまったなぁ、ここへ今日イノシシの肉が家にあったから持ってくるんだった'

In [45]:
RMeCabC(text)

[[1]]
    動詞 
"しまっ" 

[[2]]
助動詞 
  "た" 

[[3]]
  助詞 
"なぁ" 

[[4]]
記号 
"、" 

[[5]]
  名詞 
"ここ" 

[[6]]
助詞 
"へ" 

[[7]]
  名詞 
"今日" 

[[8]]
      名詞 
"イノシシ" 

[[9]]
助詞 
"の" 

[[10]]
名詞 
"肉" 

[[11]]
助詞 
"が" 

[[12]]
名詞 
"家" 

[[13]]
助詞 
"に" 

[[14]]
  動詞 
"あっ" 

[[15]]
助動詞 
  "た" 

[[16]]
  助詞 
"から" 

[[17]]
  動詞 
"持っ" 

[[18]]
助詞 
"て" 

[[19]]
  動詞 
"くる" 

[[20]]
名詞 
"ん" 

[[21]]
助動詞 
"だっ" 

[[22]]
助動詞 
  "た"